In [ ]:
from IPython.core.display import HTML, display
display(HTML('<style>.container { width:100%; !important } </style>'))

# Performance Comparison

In the following, the performance of all implemented algorithms are compared. All algorithms had a search depth of 4. For `NegamaxSearch` and `NegamaxSearchMemo`, the results are taken from the full game tests in [NegamaxTests.ipynb](NegamaxTests.ipynb). For `AlphaBetaPruning`, the results are taken from the first full game test in [AlphaBetaTests.ipynb](AlphaBetaTests.ipynb). For `AlphaBetaPruningMemo`, `AlphaBetaPruningPD` and `AlphaBetaPruningQuiescence`, a game was manually started where the algorithm played the black pieces and was given a search depth of 4. All tests were performed on a Windows PC with an AMD Ryzen 5 1600X processor (3.60 GHz, 6 cores) with 16 GB RAM. The results are summarized in the tables below.

|  | NegamaxSearch | NegamaxSearchMemo | AlphaBetaPruning
|------|------|------|------|
| Avg. search time (s) | 78.63396776083744 | 42.429948450968816 | 4.739894137932704 |
| Search time std. dev. (s) | 117.13891109514778 | 63.09473400218836 | 4.124560206430517 |  

<br>

| | AlphaBetaPruningMemo | AlphaBetaPruningPD | AlphaBetaPruningQuiescence |
|------|------|------|------|
| Avg. search time (s) | 4.433604502945803 | 4.211607594381679 | 42.23135930470058 |
| Search time std. dev. (s) | 8.600160720276488 | 9.83591474993987 | 101.16910705495255 |

<br>

These results are presented visually in the graph below.

![](Images/PerformanceDepth4.png)

These results show that memoization, alpha-beta pruning and progressive deepening reduce the time the algorithm takes to find a move, although some methods provide a more significant optimization than others.

As predicted in [AlphaBetaAlgorithm.ipynb](AlphaBetaAlgorithm.ipynb#Quiescence), quiescence search makes the search not only slower on average, but it also makes the search time much less consistent. This is because positions where many captures are possible can take much longer to evaluate than positions with few or no possible captures. However, by playing against this algorithm, it became clear that enabling quiescence search made the AI a lot smarter.

With a search depth of 4, optimizations like memoization and progressive deepening don't show their true potential. The higher the search depth, the more significant these optimizations are. Therefore, for the sake of completion, we also summarize the results from the last two full game tests in [AlphaBetaTests.ipynb](AlphaBetaTests.ipynb), which measured the performance of `AlphaBetaPruning`, `AlphaBetaPruningMemo` and `AlphaBetaPruningPD` using a search depth of 5. These results are displayed in the table below.

| | AlphaBetaPruning | AlphaBetaPruningMemo | AlphaBetaPruningPD |
|------|------|------|------|
| Avg. search time (s) | 113.90097015935021 | 99.97627545691826 | 79.51031706309078 |
| Search time std. dev. (s) | 229.23051811736033 | 194.39010036205386 | 103.75228113269658 |

<br>

These results are presented visually in the graph below.

![](Images/PerformanceDepth5.png)

These optimizations aren't as significant as memoization for pure negamax search, but they make the search time both faster and more consistent.

# Memory Usage Comparison

In the following, the memory usage of all implemented algorithms are compared. A game was played with each algorithm playing against itself using search depth of 4. To measure the memory usage, we used the `%%memit` magic command from the `memory-profiler` package (see [Pedregosa, F. \[2022\]](Bibliography.ipynb#PF22)).
All tests were performed on a Windows PC with an AMD Ryzen 5 1600X processor (3.60 GHz, 6 cores) with 16 GB RAM. The results are summarized in the tables below. Note that these values are, to a certain extent, affected by background processes. Therefore, these numbers should serve as rough estimates rather than fixed results.

|  | NegamaxSearch | NegamaxSearchMemo | AlphaBetaPruning
|------|------|------|------|
| Memory increment (MiB) | 0.01 | 0.55 | 0.02 | 

<br>

| | AlphaBetaPruningMemo | AlphaBetaPruningPD | AlphaBetaPruningQuiescence |
|------|------|------|------|
| Memory increment (MiB) | 0.05 | 0.08 | 750.25 |

<br> 

These results are presented visually in the graph below. `AlphaBetaPruningQuiescence` is left out of the graph, as its high value would make the difference between the other algorithms virtually invisible.

![](Images/MemoryUsageDepth4.png)

These results show that, as one might expect, algorithms that utilize memoization use more memory, likely because they have to keep track of a cache dictionary. That said, because the numbers are only estimates, we cannot make any definitive conclusions about the differences between the memory usage of the three algorithms that use memoization.

# Conclusion
The results of this project show that negamax search is an effective algorithm for creating a basic chess AI. Optimizations in the form of memoization, alpha-beta pruning and progressive deepening speed up the search to varying degrees. Quiescence search makes the AI much smarter, at the cost of increased search times.

The simplified evaluation described by [Michniewski, T. (2021)](Bibliography.ipynb#MT21) does a decent job of providing accurate evaluations of chess boards and moves within a realistic time frame. However, since we did not test different evaluation functions, we cannot make any definitive conclusions about its effectiveness compared to alternatives.

# Discussion and Future Prospects

Despite the implemented optimizations, the search time of the algorithms leaves something to be desired. This section covers possible reasons for this, as well as suggestions on how to improve this performance.









### Programming Language

The search algorithms are implemented in Python. In a benchmark by [Brihadiswaran, G. (2020)](Bibliography.ipynb#BG20), where the performance of Python was compared to that of C in the context of matrix multiplications, C was over 50 times as fast as Python. While this number may be different for search algorithms, it is safe to assume that implementing these algorithms in C would yield a significant performance boost. This would almost certainly make the implementation more complex, as Python offers many abstractions over C that make code more intuitive, but the performance gain would likely outweigh this downside.


### Chess Library

The chess library used in the project implements certain functionality inefficiently. From its source code (see [Fiekas, N. \[2022b\]](Bibliography.ipynb#FN22b)), we can make the following observations:
- When pushing a move to the chess board, the library updates the move counter, en passant rights and similar variables. While this is useful in most circumstances, some of these variables aren't included in our board state representation. Therefore, if a move is pushed for memoization purposes, some of these updates aren't necessary. There is no way to disable these updates, making memoization slower than it needs to be.
- As described in [AlphaBetaAlgorithm.ipynb](AlphaBetaAlgorithm.ipynb#Quiescence), checking for a check scenario is implemented inefficiently. The library implements this functionality by finding all attacking pieces on the board, finding which of these pieces attack a king, then checking if there is at least one such piece. If all that matters is whether or not a king is in check, there is no reason to find all attacking pieces.
- The library does not generate legal moves iteratively from the previous board state. When legal moves are generated, it iterates over all pieces on the board and finds the moves that these pieces can make. This could be implemented much more efficiently if the legal moves from the previous board state were accounted for; most pieces will have the same legal moves as in the previous state, so generating all moves from scratch is unnecessary.

These are just some examples. Overall, it is clear that this chess library is not optimized for performance. Using a different library or creating our own would have likely improved performance significantly.

### Delta Pruning

For quiescence search specifically, the implementation is rather naive. [Chess Programming Wiki (2020b)](Bibliography.ipynb#CPW20b) describes that a method called _delta pruning_ can be used to optimize quiescence search. As the name implies, delta pruning works by pruning branches in the search tree based on certain criteria, similar to alpha-beta pruning. Specifically, in case of a capture, delta pruning checks if the value of the captured piece, plus a small buffer value, raises the $\alpha$ value from alpha-beta pruning. If not, evaluation of this capture is skipped, as it is unlikely to improve the outcome. Delta pruning should optimize the performance of quiescence search as it needs to evaluate fewer nodes, but it is difficult to say how significant this optimization would be.

### Parallel Search

Another potential optimization is *parallel search*, also known as *multithreaded search* or *SMP search*, see [Chess programming wiki (2021b)](Bibliography.ipynb#CPW21b).
Parallel search utilizes additional processors to increase the speed of a search. Specifically, it distributes tasks among
these processors so that they can be worked on simultaneously.

For pure negamax search, implementing parallel search is trivial as different branches in the search tree do not rely on each other. One can simply
distribute branches among different processors, comparing their output and choosing the branch with the highest move score.

Implementing parallel search for negamax search with alpha-beta pruning is more complex. The $\alpha$ and $\beta$ values are constantly updated
throughout the search and therefore need to be synchronized between the processors. Similarly, if memoization is enabled,
the cache should be synchronized between the processors for optimal performance. This synchronisation makes parallelization
more difficult, but if these challenges are overcome, parallelization may yield a significant performance boost.

### Neural Networks

Lastly, the methodology used to create the AI in this project is outdated by today's standards. In 2018, Deepmind created AlphaZero, an AI that utilized neural networks to teach itself how to play chess and other board games (see [DeepMind \[2018\]](Bibliography.ipynb#DM18)). It significantly outperformed Stockfish 8, which was generally considered to be the most powerful chess AI at the time. Since then, the developers behind Stockfish have integrated neural networks into their AI as well (since Stockfish 12 released in 2020, see [Stockfish \[2020\]](Bibliography.ipynb#SC20)), suggesting that this is the future for chess AIs. Of course, this would require a large rewrite of the algorithm and would increase its complexity, but it would likely be a necessary step to take if our algorithm wanted to compete in the modern age.